In [10]:
#%matplotlib widget

#%matplotlib inline

import matplotlib
matplotlib.use('TkAgg')


import random
import pandas as pd
import cv2

import matplotlib.pyplot as plt


from sklearn.dummy import DummyRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.svm import SVR
from functools import lru_cache
import numpy as np
from scipy import ndimage


from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.decomposition import PCA

In [11]:
flatten = lambda l: [item for sublist in l for item in sublist]

def frames():
    vidcap = cv2.VideoCapture('../data/train.mp4')
    while True:
        success, image = vidcap.read()
        if success:
            yield image
        else:
            return

def crop(im, bottom=100, top=220):
    # take of top and bottom
    return im[top:im.shape[0]-bottom,:]

def tile(im, nrows=1, ncolumns=6):
    M = im.shape[0] // nrows
    N = im.shape[1] // ncolumns
    rows = []
    for y in range(0,N*ncolumns,N):
        row = []
        for x in range(0,M*nrows,M):
            row.append(im[x:x+M,y:y+N])
        rows.append(row)
    return rows

def shuffle_time_in_chunks(df, n):
    """Break df into n-lengths mini dfs"""
    assert len(df) > n*10, "doesn't meet minimum number of chunks"
    assert (len(df) % n) == 0, "all chunks equal size"

    chunk_count = len(df[0]) // n
    chunks = []
    for x in range(0, len(df), n):
        chunks.append(df[x:x + n])
    random.shuffle(chunks)
    return pd.concat(chunks, axis=0)

def quick_view(fs, frames_per_step=1, start=1000, stop=1010):
    for i, im in enumerate(fs):
        if not (start<i<stop):
            if i>stop:
                break
            continue
        if i % frames_per_step != 0:
            continue
        cv2.imshow('oned',im)
        try:
            cv2.waitKey(0)
        except KeyboardInterrupt:
            cv2.destroyAllWindows()
            break
    cv2.destroyAllWindows()

In [12]:
def cframes():
    for im in frames():
        yield crop(im)

In [13]:
#quick_view(cframes(), frames_per_step=1, start=60, stop=180)

In [19]:
cv2.destroyAllWindows()

In [38]:
assert False
# blur/focus based
def variance_of_laplacian(image):
    # compute the Laplacian of the image and then return the focus
    # measure, which is simply the variance of the Laplacian
    return cv2.Laplacian(image, cv2.CV_64F).var()
xs=[]
for i, f in enumerate(cframes()):
    tiles = tile(f,8,20)
    xs.append([variance_of_laplacian(i) for i in flatten(tiles)])
    if i % 1000 == 0:
        print('.', end='')

AssertionError: 

In [39]:
# phase correlation based
def translation_speed_via_phase(im, last_im):

    lol = cv2.phaseCorrelate(np.float32(im), np.float32(last_im))
    #??????????? which axis?
    print(lol[0][1])
    print(im.shape)
    cv2.imshow('oned',im)
    cv2.imshow('two',last_im)
    cv2.waitKey(0) # waits until a key is pressed]
    return lol[0][1]

# optical flow
def translation_speed_via_optical_flow(im, last_im):
    flow = cv2.calcOpticalFlowFarneback(last_im, im, None,
        pyr_scale=0.5,
        levels=2,
        winsize=6,
        iterations=2,
        poly_n=5,
        poly_sigma=1.2,
        flags=0)
    return cluster_velocities(flow)

def via_lk(im, last_im):
    # params for ShiTomasi corner detection
    feature_params = dict( maxCorners = 100,
                       qualityLevel = 0.3,
                       minDistance = 7,
                       blockSize = 7 )

    # Parameters for lucas kanade optical flow
    lk_params = dict( winSize  = (15,15),
                      maxLevel = 2,
                      criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))
    
    p0 = cv2.goodFeaturesToTrack(last_im, mask = None, **feature_params)
    p1, st, err = cv2.calcOpticalFlowPyrLK(last_im, im, p0, None, **lk_params)
    print(p1)

    # Create a mask image for drawing purposes
    mask = np.zeros_like(last_im)
    
    # Select good points
    good_new = p1[st==1]
    good_old = p0[st==1]

    # draw the tracks
    for i,(new,old) in enumerate(zip(good_new,good_old)):
        a,b = new.ravel()
        c,d = old.ravel()
        mask = cv2.line(mask, (a,b),(c,d), color[i].tolist(), 2)
        frame = cv2.circle(frame,(a,b),5,color[i].tolist(), -1)
    img = cv2.add(frame,mask)
    plt.imshow(img)
    plt.show()
    return 0

from sklearn.cluster import MiniBatchKMeans
def cluster_velocities(flow):
    dy,dx = flow[...,0], flow[...,1]
    shape=dy.shape
    ys,xs = np.indices(shape)
    
    # setup zenith
    xs -= shape[1] // 2
    ys += -45
    slope = dy/dx
    intercept = (-1*xs*slope)-ys
    mask = ((intercept>-350.0)&(intercept<-110.0)) ==True

    #cv2.imshow('intercept', intercept*mask)
    #cv2.imshow('intercept_mask', mask*1.0)
    #cv2.waitKey(0)
    #return 0
    plt.imshow(intercept*mask)
    plt.show()
    return 0
    

    mag, ang = cv2.cartToPolar(flow[...,0], flow[...,1])
    df = pd.DataFrame()
    df['mag'] = mag.flatten()
    df['ang'] = ang.flatten()
    df['intercept'] = intercept.flatten()
    
    print(df)
    
    df[df['intercept'].between(-300, 100)]['intercept'].plot.hist(by='intercept')
    plt.show()
    df[df['intercept'].between(-300, 100) & (df['mag']>2.0)]['mag'].plot.hist(by='mag')
    plt.show()
    return(0)
    
    
    #df[df['ang'] >0.1]['ang'].plot.hist(by='ang')
    df.replace([np.inf, -np.inf], 0, inplace=True)
    df[df['mag'] > 1]['mag'].plot.hist(by='mag')
    plt.show()
    return 0
    y : np.ndarray
    yvals = y.flatten()
    

method = translation_speed_via_optical_flow
method = via_lk

xs=[]
last_im = None
for i, im in enumerate(cframes()):
    im = cv2.cvtColor(im,cv2.COLOR_BGR2GRAY)
    if last_im is None:
        last_im = im
    #rows, columns = 3, 8
    #rows, columns = 1, 1
    #tiles = tile(im,rows,columns)
    #last_tiles = tile(last_im,rows,columns)
    #xs.append([method(im, lim) for im, lim in zip(flatten(tiles),flatten(last_tiles))])
    #xs.append([method(im,last_im)])
    method(im,last_im)
    if i+1 % 100 == 0:
        print('.', end='')
        break
    if i == 3:
        pass
      
    last_im = im

[[[264.   8.]]

 [[141.   6.]]

 [[389.   9.]]

 [[367.   9.]]

 [[274.   8.]]]


NameError: name 'color' is not defined

In [ ]:
cv2.destroyAllWindows()

In [ ]:
print(len(xs))
y = pd.read_csv('../data/train.txt', header=None)
X = pd.DataFrame(xs)

Xy = X.copy(deep=True)
Xy['y'] = y
chunksize = 240
Xy = shuffle_time_in_chunks(Xy, chunksize)

X = Xy[X.columns]
yorig = y
y = Xy[['y']]

In [ ]:
def find_testcount(test_df, fraction_testset=0.3):
    chunks = 1
    while True:
        chunks += 1
        testcount = chunksize * chunks
        if testcount/len(test_df)>fraction_testset:
            break
    print(f"using testcount={testcount}")
    return testcount
    
testcount = find_testcount(Xy, .43)

Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size=testcount, shuffle=False)
print(testcount/len(Xy))

In [ ]:
m = DummyRegressor(strategy='mean')

pipe = Pipeline([
        ('scale', StandardScaler()),
        #'reduce_dims', PCA(n_components=4)),
        ('svr', SVR())])

from sklearn.model_selection import GridSearchCV
parameters = {
    #'svr__C':[.01, .1, .3, .9, 2, 4, 10, 20],
    'svr__C':[10, 15, 20, 30, 40],
    'svr__gamma':[0.006/4, 0.006/2,0.006,0.006*2],
    }
svr = GridSearchCV(pipe, parameters, verbose=10, n_jobs=7, cv=5)

In [ ]:
m.fit(Xtrain, ytrain['y'])
svr.fit(Xtrain, ytrain['y'])
print(svr.best_estimator_)

In [ ]:
ytest_pred = svr.predict(Xtest)
ytrain_pred = svr.predict(Xtrain)
print(f"dummy test train")
print(f"{mean_squared_error(ytest, m.predict(Xtest)):0.1f} {mean_squared_error(ytest, ytest_pred):0.1f} {mean_squared_error(ytrain, ytrain_pred):0.1f}")

testing error, training error
- naive shuffle, mean dummy regressor = 68
- naive shuffle, 1x1x(varlaplace) tilessvr.predict(Xtest), SVR(C=?) = 44
- naive shuffle, 6x3x(varlaplace) tiles, SVR(C=30) = 19.4
- naive shuffle, 12x5x(varlaplace) tiles, SVR(C=30) = 10.3
- naive shuffle, test_size=.150, 26x12x(varlaplace) tiles, SVR(C=.2) = 17.9, 18.1
-    no shuffle, test_size=.150, 26x12x(varlaplace) tiles, SVR(C=.2) = 33.2, 17.7
- chunk shuffle, test_size=.147, 26x12x(varlaplace) tiles, SVR(C=.2) = 39, 16.6
- chunk shuffle, test_size=.147, 26x12x(varlaplace) tiles, SVR(C=.1) = 42, 20
- chunk shuffle, test_size=.147, 26x12x(varlaplace) tiles, SVR(C=.5) = 36, 12
- chunk shuffle, test_size=.147, 14x06x(varlaplace) tiles, SVR(C=.5) = 27, 19
- chunk shuffle, test_size=.147, 14x06x(varlaplace) tiles, SVR(C=.3) = 28.5, 21.5, 39.5
- chunk shuffle, test_size=.245, 14x06x(varlaplace) tiles, SVR(C=.3) = 50, 46, 18
# new crop
- chunk no shuffle, test_size=.245, 14x06x(varlaplace) tiles, SVR(C=.3) = 71, 39, 26.9
- chunk=5 shuffle, test_size=.367, 06x06x(varlaplace) tiles, SVR(C=.3) = 67, 30, 29.1
- chunk=5 shuffle, test_size=.491, 06x06x(varlaplace) tiles, SVR(C=.3) = 67, 30, 29.7
- chunk=5 shuffle, test_size=.73, 06x06x(varlaplace) tiles, SVR(C=.3) = 67, 32, 30.7

# Fixed tiler                                             
-                                                                dummy test train
- chunk=60 shuffle, test_size=.43, 4x10(varlaplace), SVR(C=.3) = 70.3 39.6 29.4
- chunk=60 shuffle, test_size=.43, 8x20(varlaplace), SVR(C=.3) = 71.6 31.1 22.6
- chunk=60 shuffle, test_size=.43, 8x20(varlaplace), gridsearch = 71.6 31.1 22.6
- chunk=60 focus 3x8 SVR(C=20, gamma=0.006) 64.0 13.0 2.0

## ideas
x start chunking
- increase test_size to at least .25
- accelleration limits based on testing set. This could be a post processing step.concat\
- Actually calculate the origin rather than guessing based on the crop.
- Can we get explicitly always calculate Vr
- Optical flow techniques for estimation of camera motion parameters insewer closed circuit television inspection videos
  - The reference angle is in the denominator and therefore normalizes the optical flow vector.
- Kalman:
  - https://scipy-cookbook.readthedocs.io/items/KalmanFiltering.html
- To apply kalman to blur based-must find way to unshuffle.